In [1]:
1+1


2

In [3]:
import os
from dotenv import load_dotenv
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END

# 환경 변수
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

# 도구 함수 정의
def add_two(x:str)->str:
    try:
        return str(float(x) + 2)
    except Exception:
        return "도구 실행 실패: 숫자만 일력해 주세요."

def population_lookup(city:str) -> str:
    data = {
        "서울": "9,733,509명 (2025년 기준)",
        "부산": "3,400,000명 (2025년 기준)",
    }
    return data.get(city.strip().lower(), f"{city}의 인구 정보를 찾을 수 없습니다.")

# 메시지 상태 정의
class AgentState(TypedDict):
    messages: List[BaseMessage]

# 도구 리스트 정의
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더한합니다."),
    Tool(name="PopulationLookup", func=population_lookup, description="도시 이름 입력 시 인구 정보를 반환합니다."),
]

# LLM 정의(ReAct agent는 verbose=True만 있어도 추론 흐름 출력)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=1000,
    verbose=True,
    openai_api_key=OPEN_API_KEY
)

# ReAct Agent 생성
react_agent = create_react_agent(model=llm, tools=tools, debug=True)

# LangGraph 구성(단순한 성형 흐름)
graph = StateGraph(AgentState)
graph.add_node("agent", react_agent)
graph.set_entry_point("agent")
graph.add_edge("agent", END)

# 그래프 컴파일
app = graph.compile()

# 최대 반복 횟수 설정
max_iterations = 3
recursion_limit = 2 * max_iterations + 1


# 테스트 실행
response = app.invoke(
    {
        "messages": [
            HumanMessage(content="seoul"),
        ]
    },
    {"recursion_limit": recursion_limit},
)

# 결과 출력
for m in response["messages"]:
    print(m.__class__.__name__, ":", getattr(m, "content", repr(m)))

[values] {'messages': [HumanMessage(content='seoul', additional_kwargs={}, response_metadata={}, id='2414cc0d-c82f-4ee9-b9ee-a5d5a0e75273')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1HaCyR3hsVk2rFZR2QCICOEV', 'function': {'arguments': '{"__arg1":"seoul"}', 'name': 'PopulationLookup'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 88, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpWyCQJApLuMORelF8fC3WZLoI3rP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--388bb1a5-4d54-46e6-b980-61f95c582ba6-0', tool_calls=[{'name': 'PopulationLookup', 'args': 